# Data Processing Notebook Template

Use this scaffold to build repeatable PySpark pipelines with logging, validations, and optional Delta rollbacks. Replace placeholders with your sources, business logic, and checks.


## Notebook guidelines

- Name notebooks in clear snake_case (e.g., `orders_enriched.ipynb` or `domain_orders_enriched.ipynb`); keep one primary table per notebook.
- Make runs idempotent: deterministic transforms, safe overwrites/merges, and repeatable partition logic so reruns do not create duplicates.
- Keep scopes clean: functions in snake_case, classes in PascalCase, constants UPPER_SNAKE, modules/files in snake_case; avoid implicit globals beyond parameters.
- Encapsulate helpers in small functions inside the notebook when reusable; prefer pure functions and pass Spark/DataFrames explicitly.
- Document inputs/outputs near the top and ensure the notebook owns producing one curated table/view, not many.


In [19]:
from spark_fuse.spark import create_session
from spark_fuse.utils.logging import create_progress_tracker, enable_spark_logging, log_progress as log_progress_step, console
from spark_fuse.utils.dataframe import ensure_columns, preview
from pyspark.sql import functions as F, types as T
import datetime as _dt

progress_tracker = create_progress_tracker(total_steps=10)
log = console()

def log_progress(label: str) -> None:
    log_progress_step(progress_tracker, log, label)

# Set any reusable parameters here
job_ts = _dt.datetime.now().replace(microsecond=0).isoformat()  # UTC timestamp; override if needed


> Why `functions as F` and `types as T`? Aliasing keeps chained expressions concise, matches common Spark style, and avoids polluting the global namespace with hundreds of column functions and type classes.


## Create a session

Adjust `app_name`, `master`, and configs for your environment.


In [20]:
log.log("[INFO] Starting Spark session...")
spark = create_session(
    app_name="data-processing-template",
    master="local[*]",
    # extra_configs={"spark.some.credential": "value"},
)
log.log("[INFO] Spark session ready.")
log_progress("Spark session ready")
spark


[14:37:12] [INFO] Starting Spark session...                                                         ]8;id=240131;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1197149869.py\1197149869.py]8;;\:]8;id=637731;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1197149869.py#1\1]8;;\

           [INFO] Spark session ready.                                                              ]8;id=520853;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1197149869.py\1197149869.py]8;;\:]8;id=734704;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1197149869.py#7\7]8;;\

           [INFO]  1/10 Spark session ready (+0.21s, total 0.21s)                                     ]8;id=724615;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=595624;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Start logging

Raise Spark log verbosity while you iterate so shuffle and scheduler details show up in the driver logs.


In [21]:
enable_spark_logging(spark, level="WARN")
log.log("[INFO] Spark logging enabled at WARN.")
log_progress("Logging configured")


           [INFO] Spark logging enabled at WARN.                                                     ]8;id=826303;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/692621756.py\692621756.py]8;;\:]8;id=580652;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/692621756.py#2\2]8;;\

           [INFO]  2/10 Logging configured (+0.03s, total 0.24s)                                      ]8;id=977119;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=325050;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Load relevant data

Declare input locations and load dataframes. Swap formats and options for your sources.


In [22]:
log.log("[INFO] Loading input data (dummy samples; replace with real sources)...")

orders_schema = T.StructType(
    [
        T.StructField("order_id", T.StringType(), False),
        T.StructField("order_ts", T.StringType(), False),
        T.StructField("order_total", T.DoubleType(), False),
        T.StructField("customer_id", T.StringType(), False),
    ]
)
orders_data = [
    ("O-1001", "2024-01-05", 42.50, "C001"),
    ("O-1002", "2024-01-06", 18.00, "C002"),
    ("O-1003", "2024-01-06", 120.75, "C003"),
]
orders_df = spark.createDataFrame(orders_data, schema=orders_schema)

customers_schema = T.StructType(
    [
        T.StructField("customer_id", T.StringType(), False),
        T.StructField("segment", T.StringType(), True),
        T.StructField("country", T.StringType(), True),
    ]
)
customers_data = [
    ("C001", "retail", "US"),
    ("C002", "enterprise", "CA"),
    ("C003", "retail", "UK"),
]
customers_df = spark.createDataFrame(customers_data, schema=customers_schema)

log.log("[INFO] Input data loaded.")
log_progress("Input data loaded")
log.log(f"[INFO] Orders sample:\n{preview(orders_df)}")


           [INFO] Loading input data (dummy samples; replace with real sources)...                  ]8;id=564072;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1364864045.py\1364864045.py]8;;\:]8;id=68403;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1364864045.py#1\1]8;;\

           [INFO] Input data loaded.                                                               ]8;id=563215;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1364864045.py\1364864045.py]8;;\:]8;id=30553;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1364864045.py#32\32]8;;\

           [INFO]  3/10 Input data loaded (+0.04s, total 0.28s)                                       ]8;id=266765;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=366339;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

[14:37:13] [INFO] Orders sample:                                                                   ]8;id=268947;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1364864045.py\1364864045.py]8;;\:]8;id=905819;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1364864045.py#34\34]8;;\
           rows=[{'order_id': 'O-1001', 'order_ts': '2024-01-05', 'order_total': 42.5,                             
           'customer_id': 'C001'}, {'order_id': 'O-1002', 'order_ts': '2024-01-06', 'order_total':                 
           18.0, 'customer_id': 'C002'}, {'order_id': 'O-1003', 'order_ts': '2024-01-06',                          
           'order_total': 120.75, 'customer_id': 'C003'}]                                                          
           schema=struct<order_id:string,order_ts:string,order_total:double,customer_id:string>                    

## Process data

Apply your business logic: filtering, casting, enrichment, and derived columns.


In [23]:
log.log("[INFO] Curating datasets...")
curated_orders_df = (
    orders_df
    .withColumn("order_date", F.to_date("order_ts"))
    .withColumn("order_month", F.date_format("order_date", "yyyy-MM"))
    .withColumn("processing_ts", F.lit(job_ts))
)

curated_customers_df = customers_df.select("customer_id", "segment", "country")
log.log("[INFO] Curated orders and customers dataframes ready.")
log_progress("Curated datasets ready")


           [INFO] Curating datasets...                                                              ]8;id=255504;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/2402535332.py\2402535332.py]8;;\:]8;id=910102;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/2402535332.py#1\1]8;;\

           [INFO] Curated orders and customers dataframes ready.                                   ]8;id=794460;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/2402535332.py\2402535332.py]8;;\:]8;id=132907;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/2402535332.py#10\10]8;;\

           [INFO]  4/10 Curated datasets ready (+0.64s, total 0.92s)                                  ]8;id=804904;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=227426;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Do joins

Join curated datasets and pick the right join strategy for your domain (inner/left/anti).


In [24]:
log.log("[INFO] Joining curated datasets...")
joined_df = (
    curated_orders_df.alias("o")
    .join(curated_customers_df.alias("c"), on="customer_id", how="left")
)

log.log(f"[INFO] Join complete. Sample:\n{preview(joined_df)}")
log_progress("Join complete")


           [INFO] Joining curated datasets...                                                       ]8;id=435891;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3959396089.py\3959396089.py]8;;\:]8;id=609233;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3959396089.py#1\1]8;;\

           [INFO] Join complete. Sample:                                                            ]8;id=688261;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3959396089.py\3959396089.py]8;;\:]8;id=744442;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3959396089.py#7\7]8;;\
           rows=[{'customer_id': 'C001', 'order_id': 'O-1001', 'order_ts': '2024-01-05',                           
           'order_total': 42.5, 'order_date': datetime.date(2024, 1, 5), 'order_month': '2024-01',                 
           'processing_ts': '2025-12-01T14:37:12', 'segment': 'retail', 'country': 'US'},                          
           {'customer_id': 'C002', 'order_id': 'O-1002', 'order_ts': '2024-01-06', 'order_total':                  
           18.0, 'order_date': datetime.date(2024, 1, 6), 'order_month': '2024-01',                                
           'processing_ts': '2025-12-01T14:37:12', 'segment': 'enterprise', 'country': 'CA'},                      
           {'customer_id': 'C003', 'order_id': 'O-1003', 'order_ts': '2024-01-06', 'order_total':                  
           120.75, 'order_date': datetime.date(2024, 1, 6), 'order_month': '2024-01',                              
           'processing_ts': '2025-12-01T14:37:12', 'segment': 'retail', 'country': 'UK'}]                          
           schema=struct<customer_id:string,order_id:string,order_ts:string,order_total:double,orde                
           r_date:date,order_month:string,processing_ts:string,segment:string,country:string>                      

           [INFO]  5/10 Join complete (+0.25s, total 1.17s)                                           ]8;id=343359;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=473879;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Do data tests

Add lightweight checks so issues surface early during development.


In [25]:
log.log("[INFO] Running in-memory data tests...")
# Schema/column guardrails
ensure_columns(joined_df, ["order_id", "customer_id", "order_date"])

# Null/uniqueness/data quality checks (expand as needed)
assert joined_df.filter(F.col("order_id").isNull()).count() == 0, "order_id should be populated"
assert joined_df.filter(F.col("customer_id").isNull()).count() == 0, "customer_id should be populated"
assert joined_df.dropDuplicates(["order_id"]).count() == joined_df.count(), "order_id should be unique"

# Domain-specific rule example; swap column names for your metric
invalid_states = joined_df.filter(F.col("order_total") < 0).count()
assert invalid_states == 0, f"Found {invalid_states} negative order totals"
log.log("[INFO] In-memory data tests passed.")
log_progress("In-memory data tests passed")


           [INFO] Running in-memory data tests...                                                   ]8;id=70961;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1438265954.py\1438265954.py]8;;\:]8;id=948090;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1438265954.py#1\1]8;;\

[14:37:14] [INFO] In-memory data tests passed.                                                     ]8;id=247517;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1438265954.py\1438265954.py]8;;\:]8;id=363720;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/1438265954.py#13\13]8;;\

           [INFO]  6/10 In-memory data tests passed (+0.67s, total 1.84s)                             ]8;id=38619;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=474944;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Write data & post-write tests

Persist curated results, run post-write validations, and attempt rollback when Delta Lake is available.


In [26]:
output_path = "/tmp/spark_fuse/orders_enriched_dummy"  # dummy local path; replace with real target (e.g., s3://bucket/silver/orders)
target_table = "orders_enriched_dummy"  # metastore table name if you want one registered
log.log(f"[INFO] Preparing to write dataset to {output_path}")

delta_supported = False
pre_write_version = None
output_format = "delta"
try:
    from delta.tables import DeltaTable
    delta_supported = True
    try:
        log.log(f"[INFO] Ensuring Delta table `{target_table}` is registered for {output_path} via DeltaTable API")
        (
            DeltaTable.createIfNotExists(spark)
            .tableName(target_table)
            .location(output_path)
            .addColumn("order_id", T.StringType())
            .addColumn("order_ts", T.StringType())
            .addColumn("order_total", T.DoubleType())
            .addColumn("customer_id", T.StringType())
            .addColumn("order_date", T.DateType())
            .addColumn("order_month", T.StringType())
            .addColumn("processing_ts", T.StringType())
            .addColumn("segment", T.StringType())
            .addColumn("country", T.StringType())
            .execute()
        )
    except Exception as exc:
        log.log(f"[ERROR]Could not register Delta table `{target_table}` via DeltaTable API: {exc}")

    try:
        existing_table = DeltaTable.forPath(spark, output_path)
        pre_write_version = existing_table.history(1).select("version").head()[0]
        log.log(f"[INFO] Captured pre-write Delta version {pre_write_version} for rollback.")
    except Exception:
        log.log("[INFO] No existing Delta table found at output path; rollback will be best-effort.")
except Exception:
    output_format = "parquet"
    log.log("[INFO] Delta Lake not available; falling back to parquet for write and disabling rollback.")

log.log(f"[INFO] Writing data to final path with format={output_format}...")
log_progress("Write prerequisites complete")
(
    joined_df.write
    .option("mergeSchema", "true")
    .mode("overwrite")
    .format(output_format)
    .partitionBy("order_month")
    .save(output_path)
)
log_progress("Write complete")

log.log("[INFO] Running post-write validations on persisted data...")
persisted_df = spark.read.format(output_format).load(output_path)

try:
    ensure_columns(persisted_df, ["order_id", "customer_id", "order_date", "order_month"])
    assert persisted_df.count() > 0, "Persisted dataset is empty"
    assert persisted_df.filter(F.col("order_id").isNull()).count() == 0, "order_id should be populated"
    assert persisted_df.dropDuplicates(["order_id"]).count() == persisted_df.count(), "order_id should be unique"
    invalid_persisted_states = persisted_df.filter(F.col("order_total") < 0).count()
    assert invalid_persisted_states == 0, f"Found {invalid_persisted_states} negative order totals after write"
    log.log("[INFO] Post-write validations passed.")
    log_progress("Post-write validations passed")
except Exception as exc:
    log.log(f"[ERROR]Post-write validation failed: {exc}")
    if delta_supported and pre_write_version is not None:
        try:
            log.log(f"[INFO] Attempting Delta rollback to version {pre_write_version} ...")
            spark.sql(f"RESTORE TABLE delta.`{output_path}` TO VERSION AS OF {pre_write_version}")
            log.log("[INFO] Rollback succeeded.")
        except Exception as rollback_exc:
            log.log(f"[ERROR]Rollback attempt failed: {rollback_exc}")
    else:
        log.log("[INFO] No rollback available; inspect persisted data manually.")
    raise


           [INFO] Preparing to write dataset to /tmp/spark_fuse/orders_enriched_dummy               ]8;id=191178;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py\3392025168.py]8;;\:]8;id=234470;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py#3\3]8;;\

           [INFO] Ensuring Delta table `orders_enriched_dummy` is registered for                   ]8;id=196486;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py\3392025168.py]8;;\:]8;id=919639;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py#12\12]8;;\
           /tmp/spark_fuse/orders_enriched_dummy via DeltaTable API                                                

           [ERROR]Could not register Delta table `orders_enriched_dummy` via DeltaTable API:       ]8;id=649237;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py\3392025168.py]8;;\:]8;id=563229;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py#29\29]8;;\
           [DELTA_CREATE_TABLE_WITH_DIFFERENT_PARTITIONING] The specified partitioning does not                    
           match the existing partitioning at file:/tmp/spark_fuse/orders_enriched_dummy.                          
                                                                                                                   
           == Specified ==                                                                                         
                                                                                                                   
                                                                                                                   
           == Existing ==                                                                                          
           order_month                                                                                             
                                                                                                                   

           [INFO] Captured pre-write Delta version 9 for rollback.                                 ]8;id=741309;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py\3392025168.py]8;;\:]8;id=333244;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py#34\34]8;;\

           [INFO] Writing data to final path with format=delta...                                  ]8;id=432614;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py\3392025168.py]8;;\:]8;id=36415;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py#41\41]8;;\

           [INFO]  7/10 Write prerequisites complete (+0.19s, total 2.02s)                            ]8;id=600479;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=613943;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

[14:37:16] [INFO]  8/10 Write complete (+2.12s, total 4.14s)                                          ]8;id=752640;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=724665;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

           [INFO] Running post-write validations on persisted data...                              ]8;id=436631;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py\3392025168.py]8;;\:]8;id=83761;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py#53\53]8;;\

[14:37:17] [INFO] Post-write validations passed.                                                   ]8;id=630427;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py\3392025168.py]8;;\:]8;id=102271;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/3392025168.py#63\63]8;;\

           [INFO]  9/10 Post-write validations passed (+1.27s, total 5.41s)                           ]8;id=584505;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=84366;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Stop session

Shut down the session once the job completes.


In [27]:
log.log("[INFO] Stopping Spark session.")
spark.stop()
log.log("[INFO] Spark session stopped.")
log_progress("Spark session stopped")


           [INFO] Stopping Spark session.                                                           ]8;id=720604;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/2609902046.py\2609902046.py]8;;\:]8;id=442689;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/2609902046.py#1\1]8;;\

[14:37:18] [INFO] Spark session stopped.                                                            ]8;id=677594;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/2609902046.py\2609902046.py]8;;\:]8;id=938418;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_13305/2609902046.py#3\3]8;;\

           [INFO]  10/10 Spark session stopped (+0.60s, total 6.01s)                                  ]8;id=455149;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=93290;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\